# ConversationChain
> **Adding memory (state)**

## Basic Usage

> Chains can be initialized with a Memory object, which will persist data across calls to the chain. This makes a Chain stateful.<br>
> 在calls之间持久化数据, 这将使chain变成有状态的. <br>

In [2]:
from langchain.chat_models import ChatOpenAI

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [4]:
chat = ChatOpenAI()

In [5]:
conversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())

In [6]:
conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")

'The first three colors of a rainbow are red, orange, and yellow.'

In [7]:
conversation("And the next 4?")

{'input': 'And the next 4?',
 'history': 'Human: Answer briefly. What are the first 3 colors of a rainbow?\nAI: The first three colors of a rainbow are red, orange, and yellow.',
 'response': 'The next four colors of a rainbow are green, blue, indigo, and violet.'}

> Essentially, `BaseMemory defines an interface of how langchain stores memory <br>
> It allows reading of stored data through load_memory_variables method and storing new data through save_context method. <br>

* 通过`BaseMemory.load_memory_variables` 接口存储数据<br>
* 通过`BaseMemory.save_context` 存储新数据

## Source Code parse

> `ConversationChain`是`LLMChain`的子类
```python
class ConversationChain(LLMChain):
    """Chain to have a conversation and load context from memory.

    Example:
        .. code-block:: python

            from langchain import ConversationChain, OpenAI
            conversation = ConversationChain(llm=OpenAI())
    """

    memory: BaseMemory = Field(default_factory=ConversationBufferMemory)
    """Default memory store."""
    >> PROMPT 见下文
    prompt: BasePromptTemplate = PROMPT
    """Default conversation prompt to use."""
    >> input_key 的默认值是 'input'，它在 input_keys(self) 方法中被使用。
    input_key: str = "input"  #: :meta private:
    >> output_key 则被用在 output_keys 方法中，用于指定输出字典的 key。
    output_key: str = "response"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True
    >> input_keys 属性会在 pre_inputs 方法中被调用
    @property
    def input_keys(self) -> List[str]:
        """Use this since so some prompt vars come from history."""
        return [self.input_key]

    @root_validator()
    def validate_prompt_input_variables(cls, values: Dict) -> Dict:
        """Validate that prompt input variables are consistent."""
        memory_keys = values["memory"].memory_variables
        input_key = values["input_key"]
        >> 输入键不会与内存键重叠
        if input_key in memory_keys:
            raise ValueError(
                f"The input key {input_key} was also found in the memory keys "
                f"({memory_keys}) - please provide keys that don't overlap."
            )
        prompt_variables = values["prompt"].input_variables
        expected_keys = memory_keys + [input_key]
        >> 内存键+输入键  与 prompt提示变量 必须一致, 否则抛异常
        if set(expected_keys) != set(prompt_variables):
            raise ValueError(
                "Got unexpected prompt input variables. The prompt expects "
                f"{prompt_variables}, but got {memory_keys} as inputs from "
                f"memory, and {input_key} as the normal input key."
            )
        return values

 
 ```

默认的**prompt** 接收两个键值对参数：**history** 和 **input**:

* input 是当前要处理的数据或请求。这可能是用户输入的一段文本，也可能是其他类型的数据，取决于实际应用的需求。

* history 代表了过去状态或行为的记忆，它是由 memory 对象提供的。memory 通常包含一系列先前的事件或交互，用来帮助模型生成响应或执行某些操作。<br>
  <font color=blue>例如，在一个聊天机器人应用中，history 可能包括以前的对话历史，以便生成上下文相关的回答。</font><br>


**通过这种方式，prompt 对象得以将过去的历史信息（history）和当前的输入信息（input）结合起来，生成针对特定上下文的语言模型查询。**

<font color=blue>见上文,这个是ConversationChain的默认提示模板</font>

```python
DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:"""

# 输入变量是 history 和 当前对话的输入
PROMPT = PromptTemplate(input_variables=["history", "input"], template=DEFAULT_TEMPLATE)
```

ConversationChain执行流程:<br>

1. 在 `pre_inputs`过程中，`memory` 对象会被加载，并与 `input` 变量共同构成 `prompt` 的输入。
2. 这个输入被传递给 prompt 模板，执行 `prep_prompts` 方法生成 `PromptValue`

`prep_prompts`是基类`LLMChain`的方法

```python
def prep_prompts(
    self,
    input_list: List[Dict[str, Any]],
    run_manager: Optional[CallbackManagerForChainRun] = None,
) -> Tuple[List[PromptValue], Optional[List[str]]]:
    """Prepare prompts from inputs."""
    stop = None
    if "stop" in input_list[0]:
        stop = input_list[0]["stop"]
    prompts = []
    for inputs in input_list:
        selected_inputs = {k: inputs[k] for k in self.prompt.input_variables}
        prompt = self.prompt.format_prompt(**selected_inputs)
        _colored_text = get_colored_text(prompt.to_string(), "green")
        _text = "Prompt after formatting:\n" + _colored_text
        if run_manager:
            run_manager.on_text(_text, end="\n", verbose=self.verbose)
        if "stop" in inputs and inputs["stop"] != stop:
            raise ValueError(
                "If `stop` is present in any inputs, should be present in all."
            )
        prompts.append(prompt)
    return prompts, stop
```